In [17]:
import pandas
from sklearn import datasets
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
import mlflow
from mlflow.models import infer_signature

In [18]:
# load the dataset
X,y=datasets.load_iris(return_X_y=True)

In [19]:
# splitting the data into the training and test datasets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.20)

In [34]:
# defining the model hyperparameters
params={"penalty":"l1","solver":"liblinear","max_iter":100,"multi_class":"auto","random_state":99}

In [35]:
# training the model
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\PC\Documents\Python Scripts\MLOps\MlFlow\mlflow\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(multi_class='auto', penalty='l1', random_state=99,
                   solver='liblinear')

In [36]:
# prediction on test dataset
y_pred=lr.predict(X_test)
y_pred

array([0, 2, 0, 2, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 0, 1, 1, 0, 2,
       2, 2, 2, 0, 2, 0, 2, 1])

In [37]:
# checking the model accuracy
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9

In [24]:
# MLFLOW Tracking
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [38]:
# create a new MLFlow Experiment
mlflow.set_experiment("MLFLOW initial setup")

# start an MLFLOW run
with mlflow.start_run():
    # log the hyperparameters
    mlflow.log_param("params",params)

    # logging the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training info", "Basic LR model for iris data")

    # Infer the model signature
    signature=infer_signature(X_train,lr.predict(X_train))

    # log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/11/16 02:53:58 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 2
Created version '2' of model 'tracking-quickstart'.
2024/11/16 02:53:58 INFO mlflow.tracking._tracking_service.client: 🏃 View run classy-quail-36 at: http://127.0.0.1:5000/#/experiments/842399880754009361/runs/0010e7c928994cbda73988334e568cfb.
2024/11/16 02:53:58 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842399880754009361.


In [ ]:
# trying out other parameters

# defining the model hyperparameters
params={{"solver":"newton-cg","max_iter":1000,"multi_class":"auto","random_state":1000}}

# training the model
lr=LogisticRegression(**params)
lr.fit(X_train,y_train)

c:\Users\PC\Documents\Python Scripts\MLOps\MlFlow\mlflow\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


LogisticRegression(max_iter=1000, multi_class='auto', random_state=1000,
                   solver='newton-cg')

In [40]:
# prediction on test dataset
y_pred=lr.predict(X_test)
y_pred

# checking the model accuracy
accuracy=accuracy_score(y_test,y_pred)
accuracy

0.9666666666666667

In [49]:
# start an MLFLOW run
with mlflow.start_run():
    # log the hyperparameters
    mlflow.log_param("params",params)

    # logging the accuracy metrics
    mlflow.log_metric("accuracy",accuracy)

    # set a tag that we can use to remind ourselves what this run was for
    mlflow.set_tag("Training info", "Basic LR model for iris data")

    # Infer the model signature
    signature=infer_signature(X_train,lr.predict(X_train))

    # log the model
    model_info=mlflow.sklearn.log_model(
        sk_model=lr,
        artifact_path="iris_model",
        signature=signature,
        input_example=X_train,
        registered_model_name="tracking-quickstart",
    )

Registered model 'tracking-quickstart' already exists. Creating a new version of this model...
2024/11/16 03:13:45 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: tracking-quickstart, version 3
Created version '3' of model 'tracking-quickstart'.
2024/11/16 03:13:46 INFO mlflow.tracking._tracking_service.client: 🏃 View run abrasive-owl-7 at: http://127.0.0.1:5000/#/experiments/842399880754009361/runs/da872404f70c453aa83066b5a305ccab.
2024/11/16 03:13:46 INFO mlflow.tracking._tracking_service.client: 🧪 View experiment at: http://127.0.0.1:5000/#/experiments/842399880754009361.


In [ ]:
# latest model info
model_info.model_uri

'runs:/da872404f70c453aa83066b5a305ccab/iris_model'

## Inferencing and Validating Model

In [51]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/9af3d806b62a4847a9ae754a0c982b57/iris_model'

# The model is logged with an input example. MLflow converts
# it into the serving payload format for the deployed model endpoint,
# and saves it to 'serving_input_payload.json'
serving_payload = """{
  "inputs": [
    [
      5.8,
      2.7,
      3.9,
      1.2
    ],
    [
      6.2,
      2.2,
      4.5,
      1.5
    ],
    [
      6.7,
      3.0,
      5.0,
      1.7
    ],
    [
      5.1,
      3.4,
      1.5,
      0.2
    ],
    [
      6.7,
      3.1,
      4.7,
      1.5
    ],
    [
      5.7,
      2.8,
      4.1,
      1.3
    ],
    [
      7.3,
      2.9,
      6.3,
      1.8
    ],
    [
      6.4,
      3.1,
      5.5,
      1.8
    ],
    [
      7.7,
      3.8,
      6.7,
      2.2
    ],
    [
      5.5,
      2.5,
      4.0,
      1.3
    ],
    [
      5.0,
      3.4,
      1.6,
      0.4
    ],
    [
      5.6,
      2.5,
      3.9,
      1.1
    ],
    [
      4.6,
      3.1,
      1.5,
      0.2
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      5.0,
      3.3,
      1.4,
      0.2
    ],
    [
      5.1,
      3.8,
      1.6,
      0.2
    ],
    [
      6.8,
      2.8,
      4.8,
      1.4
    ],
    [
      6.2,
      2.9,
      4.3,
      1.3
    ],
    [
      5.7,
      2.5,
      5.0,
      2.0
    ],
    [
      4.4,
      3.0,
      1.3,
      0.2
    ],
    [
      5.2,
      3.5,
      1.5,
      0.2
    ],
    [
      5.2,
      4.1,
      1.5,
      0.1
    ],
    [
      6.5,
      3.0,
      5.2,
      2.0
    ],
    [
      6.5,
      3.0,
      5.5,
      1.8
    ],
    [
      6.6,
      2.9,
      4.6,
      1.3
    ],
    [
      5.1,
      3.8,
      1.9,
      0.4
    ],
    [
      4.6,
      3.4,
      1.4,
      0.3
    ],
    [
      5.7,
      2.8,
      4.5,
      1.3
    ],
    [
      6.2,
      3.4,
      5.4,
      2.3
    ],
    [
      5.8,
      2.6,
      4.0,
      1.2
    ],
    [
      6.8,
      3.2,
      5.9,
      2.3
    ],
    [
      4.8,
      3.1,
      1.6,
      0.2
    ],
    [
      6.9,
      3.2,
      5.7,
      2.3
    ],
    [
      5.0,
      3.2,
      1.2,
      0.2
    ],
    [
      6.5,
      2.8,
      4.6,
      1.5
    ],
    [
      6.3,
      3.3,
      4.7,
      1.6
    ],
    [
      5.8,
      2.8,
      5.1,
      2.4
    ],
    [
      5.5,
      2.4,
      3.8,
      1.1
    ],
    [
      5.4,
      3.9,
      1.7,
      0.4
    ],
    [
      4.7,
      3.2,
      1.6,
      0.2
    ],
    [
      4.8,
      3.0,
      1.4,
      0.1
    ],
    [
      7.4,
      2.8,
      6.1,
      1.9
    ],
    [
      7.2,
      3.0,
      5.8,
      1.6
    ],
    [
      6.0,
      2.2,
      4.0,
      1.0
    ],
    [
      6.1,
      3.0,
      4.9,
      1.8
    ],
    [
      4.9,
      2.4,
      3.3,
      1.0
    ],
    [
      5.7,
      3.8,
      1.7,
      0.3
    ],
    [
      5.8,
      2.7,
      5.1,
      1.9
    ],
    [
      5.7,
      3.0,
      4.2,
      1.2
    ],
    [
      5.0,
      3.6,
      1.4,
      0.2
    ],
    [
      5.5,
      2.4,
      3.7,
      1.0
    ],
    [
      5.1,
      2.5,
      3.0,
      1.1
    ],
    [
      6.7,
      3.1,
      5.6,
      2.4
    ],
    [
      6.2,
      2.8,
      4.8,
      1.8
    ],
    [
      6.9,
      3.1,
      5.1,
      2.3
    ],
    [
      6.0,
      2.7,
      5.1,
      1.6
    ],
    [
      6.9,
      3.1,
      5.4,
      2.1
    ],
    [
      7.2,
      3.2,
      6.0,
      1.8
    ],
    [
      5.9,
      3.0,
      5.1,
      1.8
    ],
    [
      6.7,
      3.3,
      5.7,
      2.5
    ],
    [
      6.3,
      3.3,
      6.0,
      2.5
    ],
    [
      5.6,
      2.7,
      4.2,
      1.3
    ],
    [
      5.0,
      3.4,
      1.5,
      0.2
    ],
    [
      5.0,
      2.3,
      3.3,
      1.0
    ],
    [
      5.5,
      3.5,
      1.3,
      0.2
    ],
    [
      6.1,
      2.8,
      4.0,
      1.3
    ],
    [
      5.0,
      3.5,
      1.3,
      0.3
    ],
    [
      4.9,
      2.5,
      4.5,
      1.7
    ],
    [
      5.4,
      3.7,
      1.5,
      0.2
    ],
    [
      6.1,
      2.9,
      4.7,
      1.4
    ],
    [
      6.7,
      3.3,
      5.7,
      2.1
    ],
    [
      6.0,
      3.4,
      4.5,
      1.6
    ],
    [
      7.2,
      3.6,
      6.1,
      2.5
    ],
    [
      5.1,
      3.3,
      1.7,
      0.5
    ],
    [
      7.1,
      3.0,
      5.9,
      2.1
    ],
    [
      6.3,
      2.3,
      4.4,
      1.3
    ],
    [
      6.4,
      2.8,
      5.6,
      2.2
    ],
    [
      4.3,
      3.0,
      1.1,
      0.1
    ],
    [
      4.9,
      3.1,
      1.5,
      0.1
    ],
    [
      5.2,
      3.4,
      1.4,
      0.2
    ],
    [
      6.0,
      3.0,
      4.8,
      1.8
    ],
    [
      5.5,
      2.3,
      4.0,
      1.3
    ],
    [
      6.9,
      3.1,
      4.9,
      1.5
    ],
    [
      5.4,
      3.4,
      1.7,
      0.2
    ],
    [
      5.9,
      3.0,
      4.2,
      1.5
    ],
    [
      5.7,
      4.4,
      1.5,
      0.4
    ],
    [
      7.7,
      3.0,
      6.1,
      2.3
    ],
    [
      6.8,
      3.0,
      5.5,
      2.1
    ],
    [
      4.8,
      3.4,
      1.6,
      0.2
    ],
    [
      5.3,
      3.7,
      1.5,
      0.2
    ],
    [
      6.0,
      2.2,
      5.0,
      1.5
    ],
    [
      5.1,
      3.5,
      1.4,
      0.2
    ],
    [
      4.9,
      3.0,
      1.4,
      0.2
    ],
    [
      4.7,
      3.2,
      1.3,
      0.2
    ],
    [
      4.8,
      3.4,
      1.9,
      0.2
    ],
    [
      4.6,
      3.6,
      1.0,
      0.2
    ],
    [
      6.7,
      3.1,
      4.4,
      1.4
    ],
    [
      5.2,
      2.7,
      3.9,
      1.4
    ],
    [
      6.3,
      2.9,
      5.6,
      1.8
    ],
    [
      4.8,
      3.0,
      1.4,
      0.3
    ],
    [
      7.7,
      2.8,
      6.7,
      2.0
    ],
    [
      5.1,
      3.8,
      1.5,
      0.3
    ],
    [
      5.7,
      2.9,
      4.2,
      1.3
    ],
    [
      5.8,
      2.7,
      4.1,
      1.0
    ],
    [
      5.8,
      4.0,
      1.2,
      0.2
    ],
    [
      5.6,
      2.9,
      3.6,
      1.3
    ],
    [
      4.4,
      3.2,
      1.3,
      0.2
    ],
    [
      6.4,
      2.9,
      4.3,
      1.3
    ],
    [
      6.6,
      3.0,
      4.4,
      1.4
    ],
    [
      5.0,
      3.0,
      1.6,
      0.2
    ],
    [
      4.9,
      3.6,
      1.4,
      0.1
    ],
    [
      6.4,
      3.2,
      5.3,
      2.3
    ],
    [
      4.6,
      3.2,
      1.4,
      0.2
    ],
    [
      4.9,
      3.1,
      1.5,
      0.2
    ],
    [
      5.7,
      2.6,
      3.5,
      1.0
    ],
    [
      5.4,
      3.0,
      4.5,
      1.5
    ],
    [
      6.3,
      3.4,
      5.6,
      2.4
    ],
    [
      5.5,
      4.2,
      1.4,
      0.2
    ],
    [
      6.4,
      2.8,
      5.6,
      2.1
    ],
    [
      5.0,
      3.5,
      1.6,
      0.6
    ]
  ]
}"""

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

array([1, 1, 2, 0, 1, 1, 2, 2, 2, 1, 0, 1, 0, 2, 0, 0, 1, 1, 2, 0, 0, 0,
       2, 2, 1, 0, 0, 1, 2, 1, 2, 0, 2, 0, 1, 1, 2, 1, 0, 0, 0, 2, 2, 1,
       2, 1, 0, 2, 1, 0, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 1, 0, 1, 0, 1,
       0, 2, 0, 1, 2, 1, 2, 0, 2, 1, 2, 0, 0, 0, 2, 1, 1, 0, 1, 0, 2, 2,
       0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 2, 0, 2, 0, 1, 1, 0, 1, 0, 1, 1, 0,
       0, 2, 0, 0, 1, 1, 2, 0, 2, 0])

In [52]:
# Another way to validate model
# Load the model back for prediction as a generic python function model

loaded_model=mlflow.pyfunc.load_model('runs:/9af3d806b62a4847a9ae754a0c982b57/iris_model')
predictions=loaded_model.predict(X_test)

iris_feature_name=datasets.load_iris().feature_names

result=pandas.DataFrame(X_test,columns=iris_feature_name)
result["actual_class"]=y_test
result["predicted_class"]=predictions

In [54]:
result[:5]

sepal length (cm)  sepal width (cm)  petal length (cm)  petal width (cm)  \
0                5.1               3.7                1.5               0.4   
1                6.5               3.0                5.8               2.2   
2                4.5               2.3                1.3               0.3   
3                5.9               3.2                4.8               1.8   
4                6.4               2.7                5.3               1.9   

   actual_class  predicted_class  
0             0                0  
1             2                2  
2             0                0  
3             1                2  
4             2                2